# Loading pre-compiled csbm data

In [1]:
import sys
import torch
from csbm import MultiClassCSBM
from torch_geometric.nn import Node2Vec

In [2]:
csbm_data_list = torch.load('./csbm/csbm.pt')

In [3]:
print(len(csbm_data_list[-1].edge_index[0]))

282166


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

n = 160
csbm_zero = MultiClassCSBM(n=n, q_hom=0.25, q_het=0.05)

cuda


In [5]:
print(len(csbm_zero.edge_sources) / len(csbm_zero.X))

11.3


In [6]:
def node2vec_tuning(csbm, p, q):
    data = csbm.get_data()
    model = Node2Vec(
        data.edge_index,
        embedding_dim=128,
        walk_length=80,
        context_size=10,
        walks_per_node=10,
        num_negative_samples=1,
        p=p,
        q=q).to(device)
        
    num_workers = 4 if sys.platform == 'linux' else 0
    loader = model.loader(batch_size=16, shuffle=True, num_workers=num_workers)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.01)
    train_mask = torch.zeros(n, dtype=torch.bool)
    train_mask[:int(0.5 * n)] = 1
    test_mask = torch.zeros(n, dtype=torch.bool)
    test_mask[-int(0.5 * n):] = 1
        
    def train():
        model.train()
        total_loss = 0
        for pos_rw, neg_rw in loader:
            optimizer.zero_grad()
            loss = model.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(loader)
        
    @torch.no_grad()
    def test():
        model.eval()
        z = model()
        acc = model.test(
            train_z=z[train_mask],
            train_y=data.y[train_mask],
            test_z=z[test_mask],
            test_y=data.y[test_mask],
            max_iter=150,
        )
        return acc
    
    max_loss = 0
    max_acc = 0
    for epoch in range(150):
        loss = train()
        max_loss = max(max_loss, loss)
        acc = test()
        max_acc = max(acc, max_acc)
    print(f'p: {p}'.ljust(10) + f' q: {q}'.ljust(10) + f' Acc: {max_acc:.3f}')

In [7]:
hyperparameters = [0.25, 0.5, 1, 2, 4]
for p in hyperparameters:
    for q in hyperparameters:
        node2vec_tuning(csbm_zero, p, q)

p: 0.25q: 0.25Acc: 0.1000
p: 0.25q: 0.5 Acc: 0.1000
p: 0.25q: 1   Acc: 0.1000
p: 0.25q: 2   Acc: 0.0000


KeyboardInterrupt: 